<a href="https://colab.research.google.com/github/cbittel/Coding_Challenge/blob/main/Section1_API.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [9]:
!pip install pandas
!pip install db-sqlite3
!pip install flask pyngrok

In [10]:
from flask import Flask, request, jsonify
from pyngrok import ngrok
import threading
import io
import pandas as pd
import sqlite3

In [11]:
from google.colab import drive
drive.mount('/content/drive')


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


###SQLite3 database

In [12]:
database_path = '/content/drive/MyDrive/Data Science/Globant_tests'
database_name = f'{database_path}/historical_data.db'

In [13]:
class SQLiteDataManager:
    def __init__(self, db_name):
        """
        Initialize the SQLiteDataManager with the given database name.

        Args:
            db_name (str): The name of the SQLite database.
        """
        self.db_name = db_name

    def save_dataframe_to_sqlite(self, table_name, df):
        """
        Save a DataFrame to an SQLite table in the database.

        Args:
            table_name (str): The name of the table to save the DataFrame to.
            df (pd.DataFrame): The DataFrame to be saved.
        """
        conn = sqlite3.connect(self.db_name)
        df.to_sql(table_name, conn, if_exists='append', index=False)
        conn.close()

    def query_data_from_sqlite(self, table_name, sql_query):
        """
        Query data from an SQLite table using a custom SQL query.

        Args:
            table_name (str): The name of the table to query.
            sql_query (str): The SQL query to execute.

        Returns:
            pd.DataFrame: The result of the query as a DataFrame.
        """
        conn = sqlite3.connect(self.db_name)
        result = pd.read_sql_query(sql_query, conn)
        conn.close()
        return result

    def clear_database(self):
        """
        Query and Delete all tables from an SQLite database using SQL query.

        Returns:
            Exception: The error message.
        """
        try:
          db_connection = sqlite3.connect(database_name)
          cursor = db_connection.cursor()
          cursor.execute("SELECT name FROM sqlite_master WHERE type='table'")
          tables = cursor.fetchall()
          for table in tables:
              cursor.execute(f"DROP TABLE {table[0]}")
          db_connection.commit()
          db_connection.close()
        except Exception as e:
          return str(e)

###Flask / pyNgrok API endpoints

In [16]:
# DEfine Flask application interface
app = Flask(__name__)

def run_flask_app():
    app.run()


In [17]:

@app.route('/hello', methods=['GET'])
def hello():
    return jsonify(message="Hello from your API!")


In [18]:
@app.route('/historical_data_csv', methods=['POST'])
def historical_data_csv():
    """
    Endpoint that receives a list of three CSV files and saves them to a SQLite3 database.
    """

    try:
        files = request.files.getlist('files')

        if len(files) != 3:
            return jsonify(error="Exactly three files are required")

        # Initialize the access to data
        db_manager = SQLiteDataManager(database_name)

        # Clear the SQLite database before processing
        db_manager.clear_database()
        processed_data = []

        for file in files:
            if file.filename == '':
                return jsonify(error="One of the files is empty")
            elif file.filename == 'departments.csv':
              column_names = ['dept_id', 'dept_name']
            elif file.filename == 'jobs.csv':
              column_names = ['job_id', 'job_name']
            elif file.filename == 'hired_employees.csv':
              column_names = ['empl_id', 'empl_name', 'hired', 'dept_id', 'job_id']

            # Read the CSV file into a pandas DataFrame
            csv_data = file.read().decode('utf-8')
            df = pd.read_csv(io.StringIO(csv_data), names=column_names)   # pd.compat.StringIO(csv_data)

            # Get table name from the filename without extension
            table_name = file.filename.split('.')[0]

            batch_size = 1000
            num_batches = len(df) // batch_size + 1

            for batch_num in range(num_batches):
                batch_df = df.iloc[batch_num * batch_size : (batch_num + 1) * batch_size]
                # Process the batch - example: convert to JSON format
                processed_batch = batch_df.to_json(orient='records')
                # Process the batch - save to SQLite3 format
                db_manager.save_dataframe_to_sqlite(table_name, batch_df)

                processed_data.append(processed_batch)

            print(jsonify(processed_data.append("Data successfully processed and saved")))

        return jsonify(processed_data)
    except Exception as e:
        return jsonify(error=str(e))


###Run application and publish the endpoints

In [19]:

if __name__ == "__main__":
    # Initialize the access to data
    #db_manager = SQLiteDataManager(database_name)


    # Start the Flask app in a separate thread
    flask_thread = threading.Thread(target=run_flask_app)
    flask_thread.start()

    # Get the public URL using ngrok
    public_url = ngrok.connect(addr='5000')
    print(f'public_url: {public_url}     ---   copy this URL to the tests module "public_url" variable.')


 * Serving Flask app '__main__'
 * Debug mode: off


INFO:werkzeug:WARNING: This is a development server. Do not use it in a production deployment. Use a production WSGI server instead.
 * Running on http://127.0.0.1:5000
INFO:werkzeug:Press CTRL+C to quit


public_url: NgrokTunnel: "https://b653-35-237-3-228.ngrok.io" -> "http://localhost:5000"     ---   copy this URL to the tests module "public_url" variable.
